캘리포니아 주택 가격 데이터셋을 사용하겠습니다.

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
housing = fetch_california_housing()

In [5]:
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [6]:
input_A = keras.layers.Input(shape=[5], name='wide_input')
input_B = keras.layers.Input(shape=[6], name='deep_input')
hidden1 = keras.layers.Dense(units=30, activation='relu')(input_B)
hidden2 = keras.layers.Dense(units=30, activation='relu')(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(units=1, name='main_output')(concat)
aux_output = keras.layers.Dense(units=1, name='aux_output')(hidden2)
model = keras.Model(inputs=[input_A, input_B], outputs=[output, aux_output])

보조 출력을 추가하는 것은 아주 쉽습니다.<br>
적절한 층에 연결하고 모델의 출력 리스트에 추가하면 됩니다.

이 때 각 출력은 자신만의 손실 함수가 필요합니다.<br>
따라서 모델을 컴파일할 때 손실의 리스트를 전달해야 합니다.<br>
(하나의 손실을 전달하면 모든 출력에 하나의 손실 함수를 적용합니다.)

기본적으로 케라스는 나열된 손실을 모두 더해 최종 손실을 구해 훈련에 사용합니다.<br>
보조 출력보다 주 출력에 관심이 더 많다면<br>
주 출력의 손실에 더 많은 가중치를 부여해야 합니다.

이를 위해서 loss\_weight 파라미터에 손실 가중치 리스트를 전달해주면 됩니다.

여기서는 보조 출력을 규제를 위해서 사용하므로 메인 출력에 큰 가중치를 부여하겠습니다.

In [7]:
model.compile(loss=['mse', 'mse'], loss_weights=[0.9, 0.1],
              optimizer=keras.optimizers.SGD(learning_rate=1e-3))

In [8]:
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

모델을 훈련할 때 각 출력에 대한 레이블을 제공해야 합니다.<br>
여기서는 하위 네트워크가 유의미한 결과를 도출하는지 알아내는 것이 목적이므로<br>
둘 다 y\_valid를 전달해주겠습니다.

In [9]:
history = model.fit((X_train_A, X_train_B), (y_train, y_train), epochs=20,
                    validation_data=((X_valid_A, X_valid_B), (y_valid, y_valid)))

Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 2.1365 - main_output_loss: 1.9196 - aux_output_loss: 4.0890 - val_loss: 1.6233 - val_main_output_loss: 0.8468 - val_aux_output_loss: 8.6117
Epoch 2/20
363/363 [==============================] - 0s 1ms/step - loss: 0.8905 - main_output_loss: 0.6969 - aux_output_loss: 2.6326 - val_loss: 1.5163 - val_main_output_loss: 0.6836 - val_aux_output_loss: 9.0109
Epoch 3/20
363/363 [==============================] - 0s 975us/step - loss: 0.7429 - main_output_loss: 0.6088 - aux_output_loss: 1.9499 - val_loss: 1.4639 - val_main_output_loss: 0.6229 - val_aux_output_loss: 9.0326
Epoch 4/20
363/363 [==============================] - 0s 948us/step - loss: 0.6771 - main_output_loss: 0.5691 - aux_output_loss: 1.6485 - val_loss: 1.3388 - val_main_output_loss: 0.5481 - val_aux_output_loss: 8.4552
Epoch 5/20
363/363 [==============================] - 0s 1ms/step - loss: 0.6381 - main_output_loss: 0.5434 - aux_output_loss: 1.4911 - val_

모델을 평가하면 케라스는 개별 손실과 함께 총 손실을 반환합니다.

In [10]:
total_loss, main_loss, aux_loss = model.evaluate((X_test_A, X_test_B), (y_test, y_test))

162/162 [==============================] - 0s 696us/step - loss: 0.4668 - main_output_loss: 0.4178 - aux_output_loss: 0.9082


그리고 predict 메서드는 각 출력에 대한 예측을 반환합니다.

In [11]:
y_pred_main, y_pred_aux = model.predict((X_new_A, X_new_B))
y_pred_main, y_pred_aux

(array([[0.26762432],
        [1.980763  ],
        [3.3396285 ]], dtype=float32),
 array([[0.9593649],
        [1.9240992],
        [2.5152812]], dtype=float32))